# Inference

This Notebook can transform T2w sagittal nii.gz files into CT files.

In [1]:
from pathlib import Path

from pl_models.diffusion.diffusion import Diffusion
import utils.arguments as arguments
import torch
from torch import Tensor
import torch.nn.functional as F
import yaml
from math import ceil, floor
import numpy as np
from utils.nii import NII

model_path = "logs_diffusion3D/Diffusion_3D_T2w_CT_spine_sag/version_0/T2_to_CT_iso_diffusion_img.pt"

with open(str(model_path).replace(".pt", ".yaml")) as file:
    opt = arguments.Diffusion_Option()
    opt.__dict__ = yaml.full_load(file)
    opt.conditional_label_size = 0
    opt.conditional_dimensions = 4
    in_channel = opt.in_channel  # type: ignore
diffusion = Diffusion(opt, channel=in_channel)
dic = torch.load(model_path)
diffusion.load_state_dict(dic, strict=True)

mri_path = "data/spine/sub-111168222_sequ-402_part-inphase_dixon.nii.gz"

/home/robert/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/robert/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/robert/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/robert/.local/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
from utils.preprocessing import pad_size3D, run_model, revert_iso_to_original


def prepare_nii(mri_path):
    nii = NII.load(mri_path, False, 0)
    nii_iso = nii.rescale((1, 1, 1), verbose=True).reorient(("R", "I", "P"))
    nii_iso /= nii_iso.max()
    nii_iso.clamp_(min=0)
    nii_iso = nii_iso * 2 - 1
    arr = Tensor(nii_iso.get_array().astype(float))
    arr, padding = pad_size3D(arr)
    return arr, padding, nii, nii_iso

In [3]:
def translate(mri_path, save=True, use_cpu=False):
    arr, padding, nii, nii_iso = prepare_nii(mri_path)
    if torch.cuda.is_available() and not use_cpu:
        # If you run out of memory use max_shape= (arr.shape[-1]*arr.shape[-2]*arr.shape[-3])//2
        ct_arr = run_model(diffusion, conditional=arr, gpu=True, eta=1, w=0, steps=25, depth=0)
    else:
        print("!!! Fall back to less steps on CPU instead of 25 GPU. !!!")
        ct_arr = run_model(diffusion, conditional=arr, gpu=False, eta=1, w=0, steps=5, depth=0)
    ct_nii_iso = nii_iso.set_array(ct_arr.numpy())
    ct_nii = revert_iso_to_original(ct_nii_iso, nii, padding)
    if "_" in mri_path:
        ct_path = mri_path.rsplit("_", maxsplit=1)[0] + "_desc-translated_ct.nii.gz"
    else:
        ct_path = mri_path.rsplit(".", maxsplit=2)[0] + "_desc-translated_ct.nii.gz"
    ct_nii.save(ct_path)
    return ct_nii, ct_nii_iso

In [4]:
mri_path = "data/spine/sub-111168222_sequ-402_part-inphase_dixon.nii.gz"
ct_nii, ct_nii_iso = translate(mri_path, save=True, use_cpu=False)

print(ct_nii)
print(ct_nii_iso)

Image resampled from (0.27108434, 0.27108434, 3.3) to voxel size (1, 1, 1)
'set_array' with different dtype: from uint16 to float64
Run Diffusion; steps = 25, eta = 1.0, w = 0


sampling ddim eta=1.0: 100%|██████████| 25/25 [00:38<00:00,  1.53s/it]


'set_array' with different dtype: from float64 to float32
Save data/spine/sub-111168222_sequ-402_part-inphase_desc-translated_ct.nii.gz as float32
shp=(1024, 1024, 15); ori=('P', 'I', 'L'), zoom=(0.27108, 0.27108, 3.3), seg=False
shp=(56, 280, 280); ori=('R', 'I', 'P'), zoom=(1.0, 1.0, 1.0), seg=False


In [ ]:
mri_path = "data/spine/sub-111168222_sequ-401_part-inphase_dixon.nii.gz"
translate(mri_path, save=True)
mri_path = "data/spine/sub-111168222_sequ-403_part-inphase_dixon.nii.gz"
translate(mri_path, save=True)